### Imports

In [1]:
import numpy as np
import pandas as pd
import gc
import os
from tqdm import *
import open3d as o3d

from multiprocessing import Pool, cpu_count
from struct import *
import time
from lidar_point_pb2 import *
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import argparse
import pickle
%matplotlib notebook

### Set paths

In [2]:
ATM_PATH = "/home/sabyasachi/Projects/ati"
DATA_PATH = os.path.join(ATM_PATH, "data", "data", "datasets")

IISC_DATA_PATH = os.path.join(DATA_PATH, "IISC")
EXP_PATH = os.path.join(IISC_DATA_PATH, "2019-06-12","10-00-14-P1-6-auto-ccw_5loops_0.6_no_numba")

N_PB = 1


NEW_PCD_FOLDER = 'complete_extracted'
# LIDAR_PCD_PATH = os.path.join(EXP_PATH, "complete")
# LIDAR_CSV_PATH = os.path.join(IISC_DATA_PATH, "lidar.csv")

### Lidar pb functions

In [3]:
def read_packet(f):
    hdr = f.read(4)
    if hdr:
        #size, = struct.unpack("I", hdr)
        size, = unpack("I", hdr)
        # print(size) #Size is 488556
        #lp = ati.schema.LidarPoint()
        lp = LidarPoint()
        lp.ParseFromString(f.read(size))
        return lp

def read_frames(file_name):
    #file_name = "lidar-0"
    with open(file_name, "rb") as proto_file:
        while True:
            lp = read_packet(proto_file)

            if lp is None:
                break

            va = np.frombuffer(lp.vertical_angles, dtype=np.float64)
            ha = np.frombuffer(lp.horizontal_angles, dtype=np.float64)
            d = np.frombuffer(lp.distances, dtype=np.float64)
            i = np.frombuffer(lp.intensities, dtype=np.float64)

            beams = va.shape[0]
            num_points = d.shape[0]

            va = np.tile(va, num_points // beams).reshape(-1)
            ha = np.repeat(ha, beams).reshape(-1)
            d_v_cos = d * np.cos(va)

            frame = np.vstack([
                d_v_cos * np.sin(ha), # X
                d_v_cos * np.cos(ha), # Y
                d * np.sin(va),  # Z
                d, va, ha, i
            ]).T
            
            yield (lp.time, frame)

### Read and visualize pcd functions

In [4]:
def read_pcd(pcd_id):
    pcd_file = str(pcd_id) + ".pcd"
    pcd = o3d.io.read_point_cloud(os.path.join(LIDAR_PCD_PATH, pcd_file))
    return pcd

def draw_pcd(pcd, where='mat_3d'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.show()
    elif where is 'mat_2d':
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.show()

### Simply Visualize

In [ ]:
frm_list = []
tym_list = []
for tym, frm in tqdm_notebook(read_frames(os.path.join(EXP_PATH, 'lidar-0.pb'))):
    tym_list.append(np.ones(frm.shape[0])*tym)
    frm_list.append(frm)
gc.collect()
frm_arr = np.concatenate(frm_list)
tym_arr = np.concatenate(tym_list)
del frm_list, tym_list
gc.collect()
frm_arr.shape, tym_arr.shape

In [ ]:
df_frames = pd.DataFrame(frm_arr).rename(columns={0:'x', 1:'y', 2:'z', 3:'r', 4:'phi', 5:'tht', 6:'i'})
df_frames['t'] = tym_arr
del frm_arr, tym_arr
gc.collect()
df_frames.shape

In [ ]:
df_frames[~(df_frames['r'] == 0)].shape[0] / df_frames.shape[0]

In [ ]:
df_frames = df_frames[~(df_frames['r'] == 0)]

In [ ]:
idx_list = []
old_theta = 7
for idx, theta in tqdm_notebook(enumerate(df_frames['tht'])):
    if old_theta > theta:
        idx_list.append(idx)
    old_theta = theta
gc.collect()

In [ ]:
plt.figure()
start_idx = 0
end_idx = 5
plt.plot(np.arange(idx_list[end_idx] - idx_list[start_idx]), df_frames['tht'].iloc[idx_list[start_idx]: idx_list[end_idx]])

### Convert pb to pcd in parallel

In [5]:
def parallel_thread(idx):
    temp_df = df_frames[idx_list[idx]: idx_list[idx+1]]
    single_pcd_arr = np.array([np.array([row['x'], row['y'], row['z']]) for idx, row in temp_df.iterrows()])
    
    single_pcd = o3d.PointCloud()
    single_pcd.points = o3d.utility.Vector3dVector(single_pcd_arr)
    
    single_pcd_fname = str(pcd_name_idx + idx + 1) + ".pcd"
    single_pcd_path = os.path.join(EXP_PATH, NEW_PCD_FOLDER, single_pcd_fname)
    o3d.io.write_point_cloud(single_pcd_path, single_pcd, write_ascii=True)

In [6]:
old_theta = 7
pcd_name_idx = 0

for saby in range(N_PB):
    print('Reading frames')
    frm_list = []
    tym_list = []
    for tym, frm in tqdm_notebook(read_frames(os.path.join(EXP_PATH, 'lidar-'+ str(saby) +'.pb'))):
        tym_list.append(np.ones(frm.shape[0])*tym)
        frm_list.append(frm)
    gc.collect()
    frm_arr = np.concatenate(frm_list)
    tym_arr = np.concatenate(tym_list)
    del frm_list, tym_list
    gc.collect()
    
    print('Putting to dataframe (redundant)')
    df_frames = pd.DataFrame(frm_arr).rename(columns={0:'x', 1:'y', 2:'z', 3:'r', 4:'phi', 5:'tht', 6:'i'})
    df_frames['t'] = tym_arr
    del frm_arr, tym_arr
    gc.collect()
    
    print("Cleaning up undetected points")
    print(df_frames[~(df_frames['r'] == 0)].shape[0] / df_frames.shape[0])
    df_frames = df_frames[~(df_frames['r'] == 0)]
    
    print("Finding single revolutions")
    idx_list = []
    for idx, theta in tqdm_notebook(enumerate(df_frames['tht']), total=df_frames.shape[0]):
        if old_theta > theta:
            idx_list.append(idx)
        old_theta = theta
    gc.collect()
    
    print("Saving to pcds")
    parallel_args = range(len(idx_list) - 1)
    process_pool = Pool(cpu_count()-1)
    # process_pool = Pool(1)
    score_list = [each for each in tqdm_notebook(process_pool.imap_unordered(parallel_thread, parallel_args),
                                                 total = len(parallel_args))]
    process_pool.terminate()
    gc.collect()
    
    # pcd count in this pb file
    pcd_name_idx = pcd_name_idx + len(idx_list) - 1
    gc.collect()
    
    print('pb #{} converted to {} pcd files'.format(saby, len(idx_list)))
    print("****************************************************************")


Reading frames



Putting to dataframe (redundant)
Cleaning up undetected points
0.8994334009777696
Finding single revolutions



Saving to pcds



pb #0 converted to 880 pcd files
****************************************************************
